# Raw Excel files - Wiertsema

This notebook lists the raw Excel files provided by Wiertsema in `input_data/Wiertsema` and shows a quick preview of the first file (if any).

Run from the repository root or the notebook will attempt to detect the repo root automatically. Use the Poetry venv (e.g. `poetry shell`) so the correct Python environment and dependencies (pandas, openpyxl) are available.

In [1]:
from pathlib import Path

def find_repo_root(start=Path.cwd()):
    p = start.resolve()
    for candidate in [p] + list(p.parents):
        if (candidate / '.git').exists() or (candidate / 'pyproject.toml').exists():
            return candidate
    return p

repo_root = find_repo_root()
in_dir = repo_root / 'input_data' / 'Wiertsema'
files = sorted(in_dir.glob('*.xlsx')) + sorted(in_dir.glob('*.xls'))
print('Repository root:', repo_root)
print('Looking in:', in_dir)
print(f'Found {len(files)} Excel file(s)')
for p in files[:20]:
    print('-', p.name)

Repository root: D:\Users\jvanruitenbeek\data_validation
Looking in: D:\Users\jvanruitenbeek\data_validation\input_data\Wiertsema
Found 1 Excel file(s)
- test_87097-1_SensorData_20251007_122728.xlsx


In [2]:
import os
from pathlib import Path
from openpyxl import load_workbook

# Use correct relative path from the notebooks folder
folder = Path('../input_data/Wiertsema').resolve()
if not folder.exists():
    raise FileNotFoundError(f"Map niet gevonden: {folder}")

files = [f for f in os.listdir(folder) if f.endswith('.xlsx')]
for fname in files:
    print(f'Bestand: {fname}')
    wb = load_workbook(folder / fname, data_only=True, read_only=True)
    for sheet in wb.sheetnames:
        print(f'  Tabblad: {sheet}')
        ws = wb[sheet]
        for i, row in enumerate(ws.iter_rows(values_only=True), 1):
            print(row)
            if i >= 5:
                break
        print()

Bestand: test_87097-1_SensorData_20251007_122728.xlsx
  Tabblad: 87074-1 HB001PB01 (HB_PU0013+0_
('87074-1 HB001PB01 (HB_PU0013+0_BIT_GMW_PB1_F-6.10)', 'MSLV8_o74482', None, None)
('Timestamp', 'Water Absolute Druk (Pa)', 'Water Temperatuur (ºC)', 'Waterniveau (m NAP)')
(datetime.datetime(2024, 11, 11, 13, 0), 129120, None, -4)
(datetime.datetime(2024, 11, 11, 14, 0), 129090, None, -4.002)
(datetime.datetime(2024, 11, 11, 15, 0), 129050, None, -4.002)

  Tabblad: 87074-1 HB003PB01 (HB_PU0021+0_
('87074-1 HB003PB01 (HB_PU0021+0_KRBIB_GMW_PB1_F-5.91)', 'MSLV8_o74439', None, None)
('Timestamp', 'Water Absolute Druk (Pa)', 'Water Temperatuur (ºC)', 'Waterniveau (m NAP)')
(datetime.datetime(2024, 11, 11, 14, 0), 128920, None, -3.87)
(datetime.datetime(2024, 11, 11, 15, 0), 128660, None, -3.892)
(datetime.datetime(2024, 11, 11, 16, 0), 128630, None, -3.894)

  Tabblad: 87074-1 HB004PB01 (HB_PU0021+0_
('87074-1 HB004PB01 (HB_PU0021+0_INBIB_GMW_PB1_F-6.17)', 'MSLV8_o74371', None, None)
('Times

In [4]:
import os
from pathlib import Path
from openpyxl import load_workbook
from openpyxl.utils.datetime import from_excel
import pandas as pd
import re
from datetime import datetime

# --------- Config ----------
in_folder = Path('../input_data/Wiertsema').resolve()
out_folder = Path('../output_data/only_csv_wiertsema').resolve()
out_folder.mkdir(parents=True, exist_ok=True)
# ---------------------------

def sanitize_filename(name: str) -> str:
    """
    Make a Windows-safe filename:
    - remove '(' and ')' characters
    - replace forbidden characters <>:"/\\|?* with '_'
    - collapse whitespace
    - strip trailing spaces/dots/underscores
    """
    s = (name or "").replace("(", "").replace(")", "")
    s = re.sub(r'[<>:"/\\|?*]+', "_", s)
    s = re.sub(r"\s+", " ", s).strip()
    s = s.strip(" ._")
    return s or "series"

def norm_header(s: str) -> str:
    """
    Normalize header for fuzzy matching:
    lower, remove spaces and punctuation, remove parentheses.
    """
    if s is None:
        return ""
    s = str(s).lower()
    s = s.replace("(", "").replace(")", "")
    return re.sub(r"[^a-z0-9]+", "", s)

def first_non_empty(values):
    for v in values:
        if v not in (None, ""):
            return str(v)
    return "sheet"

def to_datetime(v, wb_epoch):
    """
    Convert a cell value to datetime if possible.
    - If it's already datetime, return it.
    - If it's a number, treat as Excel serial using workbook epoch.
    - Otherwise return None.
    """
    if isinstance(v, datetime):
        return v
    if isinstance(v, (int, float)):
        try:
            return from_excel(v, wb_epoch)
        except Exception:
            return None
    return None

def process_workbook(xlsx_path: Path):
    print(f"Bestand: {xlsx_path.name}")
    wb = load_workbook(xlsx_path, data_only=True, read_only=True)
    written = 0
    for sheet_name in wb.sheetnames:
        ws = wb[sheet_name]
        # Pull first few rows: 1) title row, 2) header row, 3+) data
        rows_iter = ws.iter_rows(values_only=True)
        try:
            title_row = next(rows_iter)
        except StopIteration:
            continue
        try:
            header_row = next(rows_iter)
        except StopIteration:
            continue

        # Build base filename from first row
        base_title = first_non_empty(title_row)
        safe_title = sanitize_filename(base_title)

        # Find columns for Timestamp and Waterniveau
        headers = [str(h) if h is not None else "" for h in header_row]
        norm_headers = [norm_header(h) for h in headers]

        # Timestamp
        ts_idx = None
        for i, nh in enumerate(norm_headers):
            if nh in ("timestamp", "time", "datetime"):
                ts_idx = i
                break

        # Waterniveau (m NAP) — be robust to variations
        wn_idx = None
        for i, nh in enumerate(norm_headers):
            # match things like "waterniveau(mnap)", "waterniveaumnap", "waterniveau m nap"
            if "waterniveau" in nh and ("mnap" in nh or "nap" in nh):
                wn_idx = i
                break

        if ts_idx is None or wn_idx is None:
            print(f"  Tabblad: {sheet_name} -> skipped (missing Timestamp or Waterniveau column)")
            continue

        # Collect data rows
        records = []
        for row in rows_iter:
            if row is None:
                continue
            # guard for ragged rows
            ts_val = row[ts_idx] if ts_idx < len(row) else None
            wn_val = row[wn_idx] if wn_idx < len(row) else None

            # Convert timestamp
            ts = to_datetime(ts_val, wb.epoch)
            if ts is None:
                # Some files store timestamps as strings like '2024-11-11 13:00'
                try:
                    ts = pd.to_datetime(ts_val, dayfirst=True, errors="coerce")
                    if pd.isna(ts):
                        continue
                except Exception:
                    continue

            # Convert waterniveau to numeric
            wn = pd.to_numeric(wn_val, errors="coerce")
            if pd.isna(wn):
                # Keep rows with NaN? Usually not; skip
                continue

            records.append((ts, wn))

        if not records:
            print(f"  Tabblad: {sheet_name} -> no valid rows")
            continue

        df = pd.DataFrame(records, columns=["Time", "Waterniveau (m NAP)"]).set_index("Time")
        # Optional: sort by time just in case
        df.sort_index(inplace=True)

        out_path = out_folder / f"{safe_title}.csv"
        # Avoid accidental overwrite if names collide across files/sheets
        if out_path.exists():
            # append a numeric suffix
            k = 1
            while True:
                candidate = out_folder / f"{safe_title}_{k}.csv"
                if not candidate.exists():
                    out_path = candidate
                    break
                k += 1

        df.to_csv(out_path, index=True, index_label="Time")
        print(f"  Tabblad: {sheet_name} -> Saved {out_path} ({len(df)} rows)")
        written += 1

    wb.close()
    print(f"Done. Saved {written} CSVs to {out_folder}\n")

# ---------- Run over folder ----------
if not in_folder.exists():
    raise FileNotFoundError(f"Map niet gevonden: {in_folder}")

xlsx_files = sorted([p for p in in_folder.iterdir() if p.suffix.lower() == ".xlsx"])
if not xlsx_files:
    print("No Excel files found in", in_folder)
else:
    for xfile in xlsx_files:
        process_workbook(xfile)

Bestand: test_83034-1_SensorData_20251007_122253.xlsx
  Tabblad: 83034-1 HB001PB01 (BE0049+00_BU -> skipped (missing Timestamp or Waterniveau column)
  Tabblad: 83034-1 HB002PB01 (BE0049+00_BI -> skipped (missing Timestamp or Waterniveau column)
  Tabblad: 83034-1 HB003PB01 (BE0049+00_IN -> Saved D:\Users\jvanruitenbeek\data_validation\output_data\only_csv_wiertsema\83034-1 HB003PB01 BE0049+00_INST_B_GMW_PB1_F-365.csv (2836 rows)
  Tabblad: 83034-1 HB005PB01 (BE115+00_BUK -> skipped (missing Timestamp or Waterniveau column)
  Tabblad: 83034-1 HB007PB01 (BE115+00_INS -> skipped (missing Timestamp or Waterniveau column)
  Tabblad: 83034-1 HB010PB01 (BE0280+00_BI -> skipped (missing Timestamp or Waterniveau column)
  Tabblad: 83034-1 HB011PB01 (BE0280+00_IN -> skipped (missing Timestamp or Waterniveau column)
  Tabblad: 83034-1 HB013PB01 (BE0341+40_BU -> skipped (missing Timestamp or Waterniveau column)
  Tabblad: 83034-1 HB015PB01 (BE0341+40_IN -> Saved D:\Users\jvanruitenbeek\data_valid